In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import json
from tqdm import tqdm_notebook as tqdm

In [ ]:
import telebot
from telebot import types
bot = telebot.TeleBot("939202590:AAFGvkADtvtsGvhABm_Br1L16H4ThaQV7z0")

In [ ]:
def get_page():
    page  = requests.get("https://1xstavka.ru/ru/live/Handball/").text  
    return BeautifulSoup(page, 'html5lib')

In [ ]:
import requests

def get_live_info():
    cookies = {'enwiki_session': '17ab96bd8ffbe8ca58a78657a918558'}

    params={
        "sports": 8,
        "count": 50,
        "antisports": 198,
        "mode": 4,
        "country": 40,
        "getEmpty": True
    }

    headers={
        "Accept": "*/*",
        "DNT": "1",
        "Referer": "https://1xbet.cr/ru/live/Handball/",
        "Sec-Fetch-Mode": "cors",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36",
        "X-Requested-With": "XMLHttpRequest"

    }
    proxies = {
#         'https': '91.213.23.110:8080'
    }

    page  = requests.get("https://1xstavka.ru/LiveFeed/Get1x2_VZip",params=params,headers=headers,proxies=proxies).text  
    return json.loads(page)

In [ ]:
try:
    with open("bet_ids.json","w") as f:
                bet_ids=json.load(f)
except:
    bet_ids={}
# get_page()


In [ ]:
from datetime import datetime
import time
nom=0

while True:
    info = get_live_info()
    if nom%5==0:
        soup = get_page()
        with open("bet_ids.json","w") as f:
            json.dump(bet_ids,f)
    
    for i in info["Value"]:
        if len(i["SC"]["PS"])>1 :
            #2ой гейм
            m1=i["SC"]["PS"][0]["Value"]["S1"]
            m2=i["SC"]["PS"][0]["Value"]["S2"]  
            if min(m1,m2)<14 and m1+m2 < 31:
                match={}
                liga = i["L"]
                country = i["CN"]
                game = i["O1"]+" - "+i["O2"]
                match_id=i["I"]
                try:
                    url = "https://1xbet.cr/ru/"+soup.find("a",href=re.compile(str(i["I"])))["href"] 
                except:
                    print("bad url")
                    url="https://1xbet.cr/ru"

                win=1
                if m1>m2:
                    win=2

                bets={
                    "TotalLess":"-",
                    "Total_player":"-",
                    "Total_player_more":"-",
                    "Total_player_less":"-",
                    "Win1":"-",
                    "Win2":"-",

                }
                for bet in i["E"]:
                    if bet["T"]==1:
                        bets["Win1"]=round(1/bet["C"],2)
                    elif bet["T"]==3:
                        bets["Win2"]==round(1/bet["C"],2)
                    elif bet["T"]==9:
                        bets["Total"]=bet["P"]
                        bets["TotalMore"]=bet["C"]  
                    elif bet["T"]==10:
                        bets["TotalLess"]=bet["C"] 

                    elif bet["T"]==11+2*(win-1):
                        bets["Total_player"]=bet["P"] 
                        bets["Total_player_less"]=bet["C"] 
                    elif bet["T"]==12+2*(win-1):
                        bets["Total_player_more"]=bet["C"] 

                match["id"]=i["I"]
                match["bet"]=bets
                match["url"]=url
                match["games"]=i["SC"]["PS"]
                match["time"]=str(datetime.now())

                with open("films.jsonlines","a") as f:
                        f.write(json.dumps(match,ensure_ascii=False)+"\n")
                        
                text=liga + "\n" + game+": ИТБ-"+str(win)+" "+str(min(m1,m2)*2-0.5) + \
                "\nДоступна ставка этого игрока на тотал больше "+ str(bets["Total_player"]) \
                +" при коэффициенте " +str(bets["Total_player_more"])+"\nВермя: "+str(datetime.now().strftime("%H:%M:%S"))
                      
                
                      
                markup = types.InlineKeyboardMarkup()
                markup.add(types.InlineKeyboardButton(text="Открыть матч", url=url))

                                                                               
                if i["I"] not in bet_ids:

                    print(game,": ИТБ-"+str(win),min(m1,m2)*2-0.5,liga)
                    print(bets["Total_player"],bets["Total_player_more"])
                    print("")

                    
                    m = bot.send_message(chat_id="@betparse",
                                                     parse_mode="HTML",
                                                     reply_markup=markup,
                                                     text=text)
                    bet_ids[i["I"]] = m.message_id
                else:                                                                               
                    bot.edit_message_text(
                        chat_id="@betparse",
                        text=text,
                        reply_markup=markup,
                        parse_mode="HTML",
                        message_id=bet_ids[i["I"]]
                    )

    time.sleep(3)
    if len(info["Value"])==0:
        time.sleep(3)
    nom+=1

